# Pocket Algorithm with FashionMNIST

Loading packages:

In [35]:
using MLDatasets, Images, TestImages

Loading the FashionMNIST dataset. It asked me for a y/n question to download the dataset, I don't know if you'll see this message.

In [21]:
train_x, train_y = FashionMNIST.traindata()
test_x,  test_y  = FashionMNIST.testdata();

This program has requested access to the data dependency FashionMNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: Fashion-MNIST
Authors: Han Xiao, Kashif Rasul, Roland Vollgraf
Website: https://github.com/zalandoresearch/fashion-mnist
License: MIT

[Han Xiao et al. 2017]
    Han Xiao, Kashif Rasul, and Roland Vollgraf.
    "Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms."
    arXiv:1708.07747

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility.



Do you want to download the dataset from ["http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz", "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz", "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz", "http://fashion

┌ Info: Downloading
│   source = http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
│   dest = /mnt/juliabox/.julia/datadeps/FashionMNIST/train-labels-idx1-ubyte.gz
│   progress = 1.0
│   time_taken = 0.1 s
│   time_remaining = 0.0 s
│   average_speed = 297.147 KiB/s
│   downloaded = 28.823 KiB
│   remaining = 0 bytes
│   total = 28.823 KiB
└ @ HTTP /opt/julia-1.0.3/share/julia/stdlib/v1.0/HTTP/src/download.jl:116
┌ Info: Downloading
│   source = http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
│   dest = /mnt/juliabox/.julia/datadeps/FashionMNIST/t10k-labels-idx1-ubyte.gz
│   progress = 1.0
│   time_taken = 0.06 s
│   time_remaining = 0.0 s
│   average_speed = 83.789 KiB/s
│   downloaded = 5.027 KiB
│   remaining = 0 bytes
│   total = 5.027 KiB
└ @ HTTP /opt/julia-1.0.3/share/julia/stdlib/v1.0/HTTP/src/download.jl:116
┌ Info: Downloading
│   source = http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-i

Using the pocket algorithm to classify images of clothing:

In [36]:
# Reducing the dimension of the input data
pocket_train_x = []
pocket_train_y = []
for i = 1:60000
    if train_y[i] == 1 || train_y[i] == 5
        push!(pocket_train_x, reshape(train_x[:,:,i], 784))
        push!(pocket_train_y, train_y[i] == 1 ? 1 : -1)
    end
end

Creating a sign function:

In [37]:
function sign(w, x)
    x = push!(copy(x), 1.0)
    return w'x > 0 ? 1 : -1
end


sign (generic function with 1 method)

Calculating errors:

In [38]:
function errors(w)
    return sum([sign(w, pocket_train_x[i])!= pocket_train_y[i] ? 1 : 0 
            for i = 1:size(pocket_train_x)[1]])/size(pocket_train_x)[1]
end

errors (generic function with 1 method)

Defining the Pocket Algorithm:

In [39]:
function pocket_algorithm(T)
    w = rand(785)
    w_hat = copy(w)
    for i = 1:T
        for j = 1:size(pocket_train_x)[1]
            if sign(w, pocket_train_x[j]) != pocket_train_y[j]
                w += pocket_train_y[j]*push!(copy(pocket_train_x[j]),1.0)
                if errors(w) < errors(w_hat)
                    w_hat = copy(w)
                end
            end
        end
    end
    return w_hat
end

pocket_algorithm (generic function with 1 method)

Finding our new weights:

In [30]:
w_new = pocket_algorithm(50);

In [31]:
size(pocket_train_y)

(12000,)

In [32]:
indicies = []
for i in 1:size(test_y)[1]
    if test_y[i] == 1 || test_y[i] == 5
        push!(indicies, (i, test_y[i]))
    end
end        

In [33]:
function check_value(i)
    println("Predicted value:", 
        sign(w_new, reshape(test_x[:,:,indicies[i][1]], 784)) == 1 ? 1 : 5, "\n")
    println("Labeled value:", test_y[indicies[i][1]], "\n")
    println("Image:")
    colorview(Gray, test_x[:,:,indicies[i][1]]')
end

check_value (generic function with 1 method)

In [34]:
check_value(1)

Predicted value:1

Labeled value:1

Image:


28×28 reinterpret(Gray{N0f8}, ::LinearAlgebra.Adjoint{Normed{UInt8,8},Array{Normed{UInt8,8},2}}):
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)  …  Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0.0)  Gray{N0f8}(0.0)
 Gray{N0f8}(0.0)  Gray{N0f8}(0.0)     Gray{N0f8}(0

All errors:
   ErrorException("error compiling #save_#28: error compiling getblob: could not load library \"/home/jrun/.julia/packages/ImageMagick/d5KBL/deps/usr/lib/libMagickWand-6.Q16.so\"\n/usr/lib/x86_64-linux-gnu/libz.so: version `ZLIB_1.2.9' not found (required by /home/jrun/.julia/packages/ImageMagick/d5KBL/deps/usr/lib/libpng16.so.16)")
   ErrorException("function save does not accept keyword arguments")


Errors encountered while saving nothing.
Fatal error:
